In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import catboost as ctb
from catboost import Pool, CatBoostClassifier
import warnings

warnings.filterwarnings('ignore')

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# # 處理訓練資料的國家資訊
geo = pd.get_dummies(train["Geography"])
train = pd.concat([train, geo], axis = 1)

# 處理訓練資料的性別資訊
gen = OneHotEncoder(sparse = False)
train["Gender"] = gen.fit_transform(train[["Gender"]])

# 選取訓練資料的變數
x = train[["CreditScore", "Gender", "Tenure", "Balance", "NumOfProducts",
           "HasCrCard", "IsActiveMember", "EstimatedSalary", "France", "Germany", "Spain"]].values
# , "France", "Germany", "Spain"
y = train["Exited"].values

# # 處理測試資料的國家資訊
geo_test = pd.get_dummies(test["Geography"])
test = pd.concat([test, geo_test], axis = 1)

# 處理測試資料的性別資訊
gen_test = OneHotEncoder(sparse = False)
test["Gender"] = gen_test.fit_transform(test[["Gender"]])

# 選取測試資料的變數
x_test = test[["CreditScore", "Gender", "Tenure", "Balance", "NumOfProducts",
           "HasCrCard", "IsActiveMember", "EstimatedSalary", "France", "Germany", "Spain"]].values


X_train, X_test, Y_train, Y_test = train_test_split(x, y,
                                                      test_size = 0.25,
                                                      random_state = 1234,
                                                      stratify = y)

# x_dataset = Pool(data = x, cat_features = [1])

classifier = CatBoostClassifier(iterations = 15, learning_rate = 0.8, depth = 10, loss_function = "CrossEntropy")

# iterations = [x for x in np.linspace(40, 100, 7)]
# learning_rate = [x for x in np.linspace(0, 1, 11)]
# depth = [x for x in np.linspace(5, 20, 16)]
# loss_function = ["Logloss", "CrossEntropy", "MultiClass"]

# random_grid = {"iterations": iterations,
#                "learning_rate": learning_rate,
#                "depth": depth,
#                "loss_function": loss_function}

# # 執行random search
# classifier_random = RandomizedSearchCV(classifier,
#                                        param_distributions = random_grid,
#                                        n_iter = 10,
#                                        cv = 2)
# classifier_random = classifier_random.fit(X_train, Y_train, cat_features = [1])
classifier = classifier.fit(X_train, Y_train)

# 預測並儲存結果
result = classifier.predict(X_test)

# result = pd.Series(result, name = "Exited")
# result = result.to_frame()

# out = pd.concat([test[["RowNumber"]], result], axis = 1)
# out.to_csv("catboost.csv")

# 印出best params
# print("best params:", classifier_random.best_params_)

# 評估訓練資料的準確度
classifier.score(X_test, Y_test)

0:	learn: 0.4577931	total: 27.9ms	remaining: 391ms
1:	learn: 0.4192472	total: 30.8ms	remaining: 200ms
2:	learn: 0.3913078	total: 60.2ms	remaining: 241ms
3:	learn: 0.3842110	total: 64.1ms	remaining: 176ms
4:	learn: 0.3675145	total: 92.6ms	remaining: 185ms
5:	learn: 0.3609866	total: 121ms	remaining: 181ms
6:	learn: 0.3562920	total: 150ms	remaining: 171ms
7:	learn: 0.3489065	total: 179ms	remaining: 157ms
8:	learn: 0.3450800	total: 207ms	remaining: 138ms
9:	learn: 0.3430557	total: 238ms	remaining: 119ms
10:	learn: 0.3400564	total: 267ms	remaining: 97.2ms
11:	learn: 0.3356685	total: 295ms	remaining: 73.8ms
12:	learn: 0.3309041	total: 321ms	remaining: 49.4ms
13:	learn: 0.3286655	total: 348ms	remaining: 24.9ms
14:	learn: 0.3271145	total: 374ms	remaining: 0us


0.8215